In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.nn import functional as F
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("LSTM-Multivariate_pollution.csv",index_col = 0)
#data['date'] = pd.to_datetime(data['date'],format='%Y-%m-%d %H:%M:%S')

In [3]:
data.dtypes

pollution    float64
dew            int64
temp         float64
press        float64
wnd_dir       object
wnd_spd      float64
snow           int64
rain           int64
dtype: object

In [4]:
data.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [5]:
wind_dir_enc = LabelEncoder()
data["wnd_dir"]= wind_dir_enc.fit_transform(data["wnd_dir"])
data["wnd_dir"] = data["wnd_dir"].astype(float)

In [44]:
data.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,2.0,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.0,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,2.0,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,2.0,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,2.0,6.25,2,0


In [6]:
values = data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled

array([[0.12977867, 0.35294118, 0.24590164, ..., 0.00229001, 0.        ,
        0.        ],
       [0.14889336, 0.36764706, 0.24590164, ..., 0.00381099, 0.        ,
        0.        ],
       [0.15995976, 0.42647059, 0.2295082 , ..., 0.00533197, 0.        ,
        0.        ],
       ...,
       [0.01006036, 0.26470588, 0.26229508, ..., 0.41399641, 0.        ,
        0.        ],
       [0.00804829, 0.26470588, 0.24590164, ..., 0.42086644, 0.        ,
        0.        ],
       [0.01207243, 0.27941176, 0.26229508, ..., 0.4262155 , 0.        ,
        0.        ]])

In [8]:
scaled.shape

(43800, 8)

In [9]:
data.isnull().any()

pollution    False
dew          False
temp         False
press        False
wnd_dir      False
wnd_spd      False
snow         False
rain         False
dtype: bool

### create sequence data

In [10]:
# split into train and test sets
n_train_hours = 365*24*4
train = scaled[:int(n_train_hours), :]
test = scaled[int(n_train_hours):, :]

In [11]:
train.shape

(35040, 8)

In [12]:
test.shape

(8760, 8)

In [13]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1,0:1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq   

In [14]:
tw = 4
train_X = []
train_Y = []

train_in_out = create_inout_sequences(train,tw)
for h in range(len(train_in_out)):
    train_x = train_in_out[h][0]
    train_y = train_in_out[h][1]
        
    train_X.append(train_x)
    train_Y.append(train_y)
    
train_X_ = np.stack(train_X) 
train_Y_ = np.stack(train_Y).flatten()    


In [17]:
train_X_.shape

(35036, 4, 8)

In [15]:
train_Y_

array([0.138833  , 0.10965795, 0.1056338 , ..., 0.02313883, 0.02012072,
       0.02313883])

In [16]:
train_Y_.shape

(35036,)

In [18]:
test_X = []
test_Y = []

test_in_out = create_inout_sequences(test,tw)
for k in range(len(test_in_out)):
    test_x = test_in_out[k][0]
    test_y = test_in_out[k][1]

    test_X.append(test_x)
    test_Y.append(test_y)
    
test_X_ = np.stack(test_X) 
test_Y_ = np.stack(test_Y).flatten()    

In [21]:
train_X_ = torch.FloatTensor(train_X_)
train_Y_ = torch.FloatTensor(train_Y_)

test_X_ = torch.FloatTensor(test_X_)
test_Y_ = torch.FloatTensor(test_Y_)

In [22]:
train_X_.shape

torch.Size([35036, 4, 8])

In [23]:
type(train_X_)

torch.Tensor

In [24]:
train_Y_.shape

torch.Size([35036])

In [25]:
test_X_.shape

torch.Size([8756, 4, 8])

In [26]:
test_Y_.shape

torch.Size([8756])

### define model

In [27]:
class LSTM_model(nn.Module):
    def __init__(self,input_size,batch_size,output_size,hidden_size,n_layers,
                drop_prob):
        super(LSTM_model,self).__init__()
        self.drop_prob = drop_prob
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.input_size = input_size
        self.batch_size = batch_size
        self.output_size = output_size
        
        self.lstm1 = nn.LSTM(input_size,hidden_size,n_layers,batch_first=True)
                           
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_size,output_size)
       
        
        
    def forward(self,input_x):
        
        #print(input_x.shape)    
        # shape of hid and cell have shape (batch_size,num_layers, hidden_size).       
        h0 = torch.zeros(self.n_layers, self.batch_size, self.hidden_size)
        c0 = torch.zeros(self.n_layers, self.batch_size, self.hidden_size)
        #print(h0.shape)
        #print(c0.shape)
        # shape of lstm_output have shape (batch_size,sequence_length, hidden_size). 
        lstm_output, (h1,c1) = self.lstm1(input_x, (h0,c0))
        
        out = self.dropout(lstm_output)  
        out = self.fc(out[:,-1,:])
        out = out.view(self.batch_size, -1)


        return out
    


### Training

In [30]:
35036/64

547.4375

In [112]:
num_epochs = 100
learning_rate = 0.001

input_size = 8
batch_size = 4
output_size = 1
#size of hidden layers
hidden_size = 32
drop_prob = 0.5
num_layers = 2

train_loader = DataLoader(list(zip(train_X_,train_Y_)), batch_size=batch_size,shuffle=False, drop_last=True)
test_loader = DataLoader(list(zip(test_X_,test_Y_)), batch_size=batch_size, shuffle=False, drop_last=True) 

model = LSTM_model(input_size, batch_size, output_size, hidden_size, num_layers,drop_prob)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    
    model.train()
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)
        # obtain the loss function
        loss = criterion(outputs,y_batch)
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        if epoch % 5 == 0:
          print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))


Epoch: 0, loss: 0.00038
Epoch: 0, loss: 0.00142
Epoch: 0, loss: 0.00098
Epoch: 0, loss: 0.00175
Epoch: 0, loss: 0.00050
Epoch: 0, loss: 0.00604
Epoch: 0, loss: 0.00747
Epoch: 0, loss: 0.00379
Epoch: 0, loss: 0.00176
Epoch: 0, loss: 0.00634
Epoch: 0, loss: 0.00439
Epoch: 0, loss: 0.00861
Epoch: 0, loss: 0.00720
Epoch: 0, loss: 0.01221
Epoch: 0, loss: 0.00981
Epoch: 0, loss: 0.00899
Epoch: 0, loss: 0.00712
Epoch: 0, loss: 0.00345
Epoch: 0, loss: 0.00144
Epoch: 0, loss: 0.00230
Epoch: 0, loss: 0.00324
Epoch: 0, loss: 0.00151
Epoch: 0, loss: 0.00113
Epoch: 0, loss: 0.00346
Epoch: 0, loss: 0.00119
Epoch: 0, loss: 0.00336
Epoch: 0, loss: 0.00273
Epoch: 0, loss: 0.00220
Epoch: 0, loss: 0.01198
Epoch: 0, loss: 0.00710
Epoch: 0, loss: 0.00140
Epoch: 0, loss: 0.00042
Epoch: 0, loss: 0.00418
Epoch: 0, loss: 0.00413
Epoch: 0, loss: 0.02200
Epoch: 0, loss: 0.03461
Epoch: 0, loss: 0.00224
Epoch: 0, loss: 0.00423
Epoch: 0, loss: 0.02239
Epoch: 0, loss: 0.02750
Epoch: 0, loss: 0.01344
Epoch: 0, loss: 

Epoch: 0, loss: 0.00128
Epoch: 0, loss: 0.00031
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00210
Epoch: 0, loss: 0.00023
Epoch: 0, loss: 0.00818
Epoch: 0, loss: 0.00258
Epoch: 0, loss: 0.00070
Epoch: 0, loss: 0.00064
Epoch: 0, loss: 0.00029
Epoch: 0, loss: 0.00087
Epoch: 0, loss: 0.00089
Epoch: 0, loss: 0.00187
Epoch: 0, loss: 0.00020
Epoch: 0, loss: 0.00073
Epoch: 0, loss: 0.00049
Epoch: 0, loss: 0.00077
Epoch: 0, loss: 0.00144
Epoch: 0, loss: 0.00083
Epoch: 0, loss: 0.00025
Epoch: 0, loss: 0.00060
Epoch: 0, loss: 0.00273
Epoch: 0, loss: 0.00119
Epoch: 0, loss: 0.00037
Epoch: 0, loss: 0.00044
Epoch: 0, loss: 0.00128
Epoch: 0, loss: 0.00869
Epoch: 0, loss: 0.00769
Epoch: 0, loss: 0.01153
Epoch: 0, loss: 0.01151
Epoch: 0, loss: 0.00206
Epoch: 0, loss: 0.00933
Epoch: 0, loss: 0.00628
Epoch: 0, loss: 0.00098
Epoch: 0, loss: 0.00064
Epoch: 0, loss: 0.00071
Epoch: 0, loss: 0.11204
Epoch: 0, loss: 0.02307
Epoch: 0, loss: 0.00685
Epoch: 0, loss: 0.00633
Epoch: 0, loss: 

Epoch: 0, loss: 0.00094
Epoch: 0, loss: 0.00077
Epoch: 0, loss: 0.00024
Epoch: 0, loss: 0.00075
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00012
Epoch: 0, loss: 0.00135
Epoch: 0, loss: 0.00009
Epoch: 0, loss: 0.00139
Epoch: 0, loss: 0.00140
Epoch: 0, loss: 0.00219
Epoch: 0, loss: 0.00311
Epoch: 0, loss: 0.00407
Epoch: 0, loss: 0.00123
Epoch: 0, loss: 0.00263
Epoch: 0, loss: 0.00742
Epoch: 0, loss: 0.00512
Epoch: 0, loss: 0.01146
Epoch: 0, loss: 0.00900
Epoch: 0, loss: 0.00588
Epoch: 0, loss: 0.00286
Epoch: 0, loss: 0.01652
Epoch: 0, loss: 0.00620
Epoch: 0, loss: 0.00293
Epoch: 0, loss: 0.00187
Epoch: 0, loss: 0.00246
Epoch: 0, loss: 0.00082
Epoch: 0, loss: 0.00118
Epoch: 0, loss: 0.00150
Epoch: 0, loss: 0.00021
Epoch: 0, loss: 0.00028
Epoch: 0, loss: 0.00181
Epoch: 0, loss: 0.00148
Epoch: 0, loss: 0.00259
Epoch: 0, loss: 0.00220
Epoch: 0, loss: 0.00057
Epoch: 0, loss: 0.00042
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00440
Epoch: 0, loss: 0.00476
Epoch: 0, loss: 0.00880
Epoch: 0, loss: 

Epoch: 0, loss: 0.00164
Epoch: 0, loss: 0.00033
Epoch: 0, loss: 0.00068
Epoch: 0, loss: 0.00061
Epoch: 0, loss: 0.00558
Epoch: 0, loss: 0.00759
Epoch: 0, loss: 0.00499
Epoch: 0, loss: 0.00554
Epoch: 0, loss: 0.00296
Epoch: 0, loss: 0.00408
Epoch: 0, loss: 0.00646
Epoch: 0, loss: 0.00309
Epoch: 0, loss: 0.01125
Epoch: 0, loss: 0.00387
Epoch: 0, loss: 0.00143
Epoch: 0, loss: 0.00058
Epoch: 0, loss: 0.00043
Epoch: 0, loss: 0.00298
Epoch: 0, loss: 0.00662
Epoch: 0, loss: 0.00246
Epoch: 0, loss: 0.00654
Epoch: 0, loss: 0.00580
Epoch: 0, loss: 0.00222
Epoch: 0, loss: 0.00316
Epoch: 0, loss: 0.00061
Epoch: 0, loss: 0.00027
Epoch: 0, loss: 0.00023
Epoch: 0, loss: 0.00193
Epoch: 0, loss: 0.00058
Epoch: 0, loss: 0.00012
Epoch: 0, loss: 0.00044
Epoch: 0, loss: 0.00483
Epoch: 0, loss: 0.00393
Epoch: 0, loss: 0.00189
Epoch: 0, loss: 0.00902
Epoch: 0, loss: 0.00137
Epoch: 0, loss: 0.00132
Epoch: 0, loss: 0.00277
Epoch: 0, loss: 0.00078
Epoch: 0, loss: 0.00074
Epoch: 0, loss: 0.00198
Epoch: 0, loss: 

Epoch: 0, loss: 0.00039
Epoch: 0, loss: 0.00176
Epoch: 0, loss: 0.00183
Epoch: 0, loss: 0.00246
Epoch: 0, loss: 0.00208
Epoch: 0, loss: 0.00077
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00228
Epoch: 0, loss: 0.00825
Epoch: 0, loss: 0.00434
Epoch: 0, loss: 0.00425
Epoch: 0, loss: 0.00385
Epoch: 0, loss: 0.00229
Epoch: 0, loss: 0.01297
Epoch: 0, loss: 0.00247
Epoch: 0, loss: 0.00272
Epoch: 0, loss: 0.00474
Epoch: 0, loss: 0.00554
Epoch: 0, loss: 0.00352
Epoch: 0, loss: 0.00394
Epoch: 0, loss: 0.00185
Epoch: 0, loss: 0.00167
Epoch: 0, loss: 0.00062
Epoch: 0, loss: 0.00043
Epoch: 0, loss: 0.00168
Epoch: 0, loss: 0.00060
Epoch: 0, loss: 0.00032
Epoch: 0, loss: 0.00204
Epoch: 0, loss: 0.00217
Epoch: 0, loss: 0.00136
Epoch: 0, loss: 0.00025
Epoch: 0, loss: 0.00080
Epoch: 0, loss: 0.00131
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00081
Epoch: 0, loss: 0.00278
Epoch: 0, loss: 0.00028
Epoch: 0, loss: 0.00048
Epoch: 0, loss: 0.00084
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00021
Epoch: 0, loss: 

...

Epoch: 0, loss: 0.00142
Epoch: 0, loss: 0.00098
Epoch: 0, loss: 0.00097
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00168
Epoch: 0, loss: 0.00043
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00028
Epoch: 0, loss: 0.00078
Epoch: 0, loss: 0.00023
Epoch: 0, loss: 0.00014
Epoch: 0, loss: 0.00061
Epoch: 0, loss: 0.00134
Epoch: 0, loss: 0.00128
Epoch: 0, loss: 0.00079
Epoch: 0, loss: 0.00157
Epoch: 0, loss: 0.00314
Epoch: 0, loss: 0.00507
Epoch: 0, loss: 0.00142
Epoch: 0, loss: 0.00309
Epoch: 0, loss: 0.00098
Epoch: 0, loss: 0.00081
Epoch: 0, loss: 0.00004
Epoch: 0, loss: 0.00060
Epoch: 0, loss: 0.00022
Epoch: 0, loss: 0.00080
Epoch: 0, loss: 0.00058
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00008
Epoch: 0, loss: 0.00080
Epoch: 0, loss: 0.00050
Epoch: 0, loss: 0.00012
Epoch: 0, loss: 0.00168
Epoch: 0, loss: 0.00106
Epoch: 0, loss: 0.00049
Epoch: 0, loss: 0.00025
Epoch: 0, loss: 0.00097
Epoch: 0, loss: 0.00271
Epoch: 0, loss: 0.00334
Epoch: 0, loss: 

Epoch: 0, loss: 0.00031
Epoch: 0, loss: 0.00089
Epoch: 0, loss: 0.00036
Epoch: 0, loss: 0.00026
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00063
Epoch: 0, loss: 0.00012
Epoch: 0, loss: 0.00036
Epoch: 0, loss: 0.00055
Epoch: 0, loss: 0.00009
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00012
Epoch: 0, loss: 0.00015
Epoch: 0, loss: 0.00009
Epoch: 0, loss: 0.00067
Epoch: 0, loss: 0.00014
Epoch: 0, loss: 0.00008
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00007
Epoch: 0, loss: 0.00014
Epoch: 0, loss: 0.00020
Epoch: 0, loss: 0.00044
Epoch: 0, loss: 0.00017
Epoch: 0, loss: 0.00008
Epoch: 0, loss: 0.00016
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00064
Epoch: 0, loss: 0.00451
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00006
Epoch: 0, loss: 0.00016
Epoch: 0, loss: 0.00018
Epoch: 0, loss: 0.00114
Epoch: 0, loss: 0.00027
Epoch: 0, loss: 0.00060
Epoch: 0, loss: 0.00112
Epoch: 0, loss: 0.00050
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00011
Epoch: 0, loss: 

Epoch: 0, loss: 0.00121
Epoch: 0, loss: 0.00039
Epoch: 0, loss: 0.00025
Epoch: 0, loss: 0.00029
Epoch: 0, loss: 0.00028
Epoch: 0, loss: 0.00022
Epoch: 0, loss: 0.00154
Epoch: 0, loss: 0.00052
Epoch: 0, loss: 0.00026
Epoch: 0, loss: 0.00034
Epoch: 0, loss: 0.00042
Epoch: 0, loss: 0.00030
Epoch: 0, loss: 0.00009
Epoch: 0, loss: 0.00077
Epoch: 0, loss: 0.00071
Epoch: 0, loss: 0.00029
Epoch: 0, loss: 0.00004
Epoch: 0, loss: 0.00046
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00019
Epoch: 0, loss: 0.00021
Epoch: 0, loss: 0.00026
Epoch: 0, loss: 0.00203
Epoch: 0, loss: 0.00017
Epoch: 0, loss: 0.00013
Epoch: 0, loss: 0.00068
Epoch: 0, loss: 0.00017
Epoch: 0, loss: 0.00041
Epoch: 0, loss: 0.00021
Epoch: 0, loss: 0.00046
Epoch: 0, loss: 0.00087
Epoch: 0, loss: 0.00210
Epoch: 0, loss: 0.00135
Epoch: 0, loss: 0.00125
Epoch: 0, loss: 0.00982
Epoch: 0, loss: 0.00029
Epoch: 0, loss: 0.00049
Epoch: 0, loss: 0.00041
Epoch: 0, loss: 0.00026
Epoch: 0, loss: 0.00016
Epoch: 0, loss: 0.00069
Epoch: 0, loss: 

Epoch: 5, loss: 0.01379
Epoch: 5, loss: 0.00084
Epoch: 5, loss: 0.00021
Epoch: 5, loss: 0.01149
Epoch: 5, loss: 0.00432
Epoch: 5, loss: 0.00744
Epoch: 5, loss: 0.00373
Epoch: 5, loss: 0.00186
Epoch: 5, loss: 0.00128
Epoch: 5, loss: 0.00437
Epoch: 5, loss: 0.00495
Epoch: 5, loss: 0.00561
Epoch: 5, loss: 0.00820
Epoch: 5, loss: 0.00192
Epoch: 5, loss: 0.00175
Epoch: 5, loss: 0.00159
Epoch: 5, loss: 0.00136
Epoch: 5, loss: 0.00109
Epoch: 5, loss: 0.00082
Epoch: 5, loss: 0.00047
Epoch: 5, loss: 0.00098
Epoch: 5, loss: 0.00040
Epoch: 5, loss: 0.00051
Epoch: 5, loss: 0.00019
Epoch: 5, loss: 0.00010
Epoch: 5, loss: 0.00046
Epoch: 5, loss: 0.00006
Epoch: 5, loss: 0.00009
Epoch: 5, loss: 0.00036
Epoch: 5, loss: 0.00017
Epoch: 5, loss: 0.00010
Epoch: 5, loss: 0.00024
Epoch: 5, loss: 0.00030
Epoch: 5, loss: 0.00043
Epoch: 5, loss: 0.00011
Epoch: 5, loss: 0.00025
Epoch: 5, loss: 0.00020
Epoch: 5, loss: 0.00090
Epoch: 5, loss: 0.00045
Epoch: 5, loss: 0.00012
Epoch: 5, loss: 0.00046
Epoch: 5, loss: 

KeyboardInterrupt: 

In [ ]:
model.eval()

Y_pred = []
Y_true = []

for x_batch, y_true in test_loader:
    y_pred = model(x_batch).flatten()
    y_pred = y_pred.detach().numpy().tolist()
    Y_pred.append(y_pred)
    Y_true.append(y_true)
    

Y_pred_ = np.array([item for sublist in Y_pred for item in sublist])
Y_true_  = np.array([item for sublist in Y_true for item in sublist])
    

In [ ]:
predict_array = np.zeros(shape=(len(Y_pred_), 8) )
true_array = np.zeros(shape=(len(Y_true_), 8) )

# put the predicted values in the right field
predict_array[:,0] = Y_pred_
true_array[:,0] = Y_true_

# inverse transform and then select the right field
Y_prediction = scaler.inverse_transform(predict_array)[:,0]
Y_true = scaler.inverse_transform(true_array)[:,0]



plt.plot(Y_prediction)
plt.plot(Y_true)
plt.suptitle('Time-Series Prediction')
plt.show()

In [ ]:
Y_true_[-20:]

In [ ]:
test[-20:]

In [ ]:
y_true